In [1]:
from cProfile import label
import re
import hydra
from scipy import rand
import torch.utils.data as data
import torch
from torchvision import datasets, transforms
from PIL import Image
import json
import os
import zipfile
import numpy as np
import random

### todo file the config, complete
class VideoDataset(data.Dataset):
    def __init__(self,args):
        super(VideoDataset,self).__init__()
        self.args = args
        self.params = self.args[self.args.task]
        self.an_classes = self.params.an_classes
        if self.args.dataset == 'ActivityNet':
            self.extract_path = self.args.data_paths.ActivityNet.extracted
            self.annotation = self.args.data_paths.ActivityNet.annotation

        self.annot = json.load(open(self.annotation,'r'))
        self.video_dict = {}
        # get task specific data dict, for each class choose one query and 1 or 5 support videos
        self.get_fs_data(self.args.task,self.video_dict)

    def get_fs_data(self,task,video_dict):
        for video_class in self.an_classes:
            video_dict[video_class] = {}
            for video_name in os.listdir(os.path.join(self.extract_path,video_class)):
                if self.annot['database'][video_name]['subset'] == task:
                    video_dict[video_class][video_name] = self.annot['database'][video_name]

    def get_video(self,video_list,video_class):
        videos = []
        for video_name in video_list:
            video_path = os.path.join(self.extract_path,video_class,video_name)
            frames = []
            for frame in sorted(os.listdir(video_path)):
                with Image.open(os.path.join(video_path,frame)) as i:
                    i.load()
                    frames.append(np.asarray(i))
            frames = np.stack(frames,axis=0)
            videos.append(frames)
        return videos
    
    def get_labels(self,video_list,video_class):
        labels = []
        for video_name in video_list:
            labels.append(self.video_dict[video_class][video_name]['annotations'][0]['segment'])
        return labels

    #todo: may be have repeat class,seed infect the random?
    def __getitem__(self, index):
        # get class name
        videos = []
        labels = []
        video_classes = random.sample(self.an_classes, self.args.way)
        for video_class in video_classes:
            video_list = random.sample(self.video_dict[video_class].keys(), self.args.shot + self.args.query_per_class)
            # get query and support video,only one way
            videos = self.get_video(video_list,video_class)
            labels = self.get_labels(video_list,video_class)
        query_video = videos[:self.args.query_per_class]
        support_video = videos[self.args.query_per_class:]
        query_label = labels[:self.args.query_per_class]
        support_label = labels[self.args.query_per_class:]

        return {'qv':query_video,'sv':support_video,'ql':query_label,'sl':support_label}

    def __len__(self):
        return 9999999


/home/wangqiurui/anaconda3/envs/strm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
p = '/data1/wangqiurui/code/datasets/ActivityNet/anet1.3_tsp_filtered.json'

In [3]:
an = json.load(open(p,'r'))

In [2]:
@hydra.main(config_path="config", config_name="config",version_base=None)
def main(cfg):
    a = VideoDataset(cfg)
    d = a.__getitem__(1)
    qv = d['qv']


In [3]:

main()

usage: ipykernel_launcher.py [--help] [--hydra-help] [--version]
                             [--cfg {job,hydra,all}] [--resolve]
                             [--package PACKAGE] [--run] [--multirun]
                             [--shell-completion] [--config-path CONFIG_PATH]
                             [--config-name CONFIG_NAME]
                             [--config-dir CONFIG_DIR]
                             [--experimental-rerun EXPERIMENTAL_RERUN]
                             [--info [{all,config,defaults,defaults-tree,plugins,searchpath}]]
                             [overrides ...]
ipykernel_launcher.py: error: unrecognized arguments: --f=/home/wangqiurui/.local/share/jupyter/runtime/kernel-v2-50898AU2Crr0mRlo1.json


SystemExit: 2

/home/wangqiurui/anaconda3/envs/strm/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3556: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
